# WikiSQL Training Data Preparation

This notebook prepares training data for SQL generation using the WikiSQL dataset.

**Dataset:** [WikiSQL](https://huggingface.co/datasets/wikisql) - 80k+ examples of natural language questions → SQL queries

**Format:** `[TASK: SQL] Table: <schema> | Question: <question> → SELECT query`

In [ ]:
%pip install datasets tiktoken pandas

## Configuration

In [ ]:
# How many training examples to use
NUM_SAMPLES = 10000  # Start with 10k, can increase to 20k or 80k

# Context window settings
CONTEXT_LENGTH = 1024

# Output directory
OUTPUT_DIR = "./data/sql_training_pairs"

## Step 1: Load WikiSQL Dataset

In [ ]:
from datasets import load_dataset
import pandas as pd
import tiktoken

print("Loading WikiSQL dataset...")
# Use htriedman/wikisql - already formatted as input/output pairs
ds = load_dataset("htriedman/wikisql")

print(f"\nDataset splits:")
print(f"  Train: {len(ds['train'])} examples")
print(f"  Validation: {len(ds['validation'])} examples")
print(f"  Test: {len(ds['test'])} examples")

# Convert to pandas for easier manipulation
df = ds['train'].to_pandas()
print(f"\nUsing first {NUM_SAMPLES} training examples")

## Step 2: Explore Dataset Format

In [ ]:
# Look at first example
example = df.iloc[0]

print("Example WikiSQL entry:")
print("="*60)
print(f"Instruction: {example['instruction']}")
print(f"Input (question): {example['input']}")
print(f"Output (SQL): {example['output']}")

<cell_type>markdown</cell_type>## Step 3: Format Training Examples with Task Prefix

In [ ]:
def format_sql_example(row):
    """
    Format: [TASK: SQL] Question: what is x? → SQL query
    
    htriedman/wikisql already has:
      - instruction: "Translate the following into a SQL query"
      - input: natural language question
      - output: SQL query
    """
    question = row['input']
    sql_query = row['output']
    
    # Format with task prefix
    prompt = f"[TASK: SQL] Question: {question}"
    completion = sql_query
    
    # Concatenate input + output for next-token prediction
    full_text = f"{prompt} → {completion}"
    
    return full_text

# Test formatting
example_text = format_sql_example(df.iloc[0])
print("Formatted training example:")
print("="*60)
print(example_text)
print("="*60)
print(f"Length: {len(example_text)} characters")

<cell_type>markdown</cell_type>## Step 4: Create Training Pairs and Tokenize

In [ ]:
<cell_type>markdown</cell_type>## Step 5: Inspect Training Examples

<cell_type>markdown</cell_type>## Step 6: Save as Binary Files for C-Transformer

In [ ]:
import tiktoken
from pathlib import Path

# Initialize tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# Format all examples and tokenize
print(f"Processing {NUM_SAMPLES} examples...")

training_examples = []
skipped = 0

for idx in range(min(NUM_SAMPLES, len(df))):
    row = df.iloc[idx]
    
    # Format example
    formatted_text = format_sql_example(row)
    
    # Tokenize
    tokens = tokenizer.encode(formatted_text)
    
    # Skip if too long (need room for input + target)
    if len(tokens) > CONTEXT_LENGTH:
        skipped += 1
        continue
    
    training_examples.append({
        'text': formatted_text,
        'tokens': tokens,
        'ctx_len': len(tokens),  # Actual length (variable)
        'example_id': idx
    })
    
    if (idx + 1) % 1000 == 0:
        print(f"  Processed {idx + 1}/{NUM_SAMPLES}")

print(f"\nCreated {len(training_examples)} training examples")
print(f"Skipped {skipped} examples (too long)")

# Convert to DataFrame
df_train = pd.DataFrame(training_examples)

# Show statistics
print(f"\nToken length statistics:")
print(df_train['ctx_len'].describe())

<cell_type>markdown</cell_type>## Step 7: Verify Binary Files

In [ ]:
# Show a few examples
print("Sample Training Examples:")
print("="*80)

for i in range(min(5, len(df_train))):
    example = df_train.iloc[i]
    print(f"\nExample {i}:")
    print(f"  Text: {example['text']}")
    print(f"  Tokens: {example['ctx_len']}")
    print(f"  First 10 token IDs: {example['tokens'][:10]}")
    print("-"*80)

<cell_type>markdown</cell_type>## Step 8: Save Metadata

In [ ]:
import struct
from pathlib import Path

# Create output directory
output_dir = Path(OUTPUT_DIR)
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Saving {len(df_train)} training pairs to {output_dir}")
print("="*60)

for idx, row in df_train.iterrows():
    tokens = row['tokens']
    total_tokens = len(tokens)

    # Need at least two tokens to form an input/target pair
    if total_tokens < 2:
        continue

    # ctx_len counts how many input positions we keep (reserve one for the next-token target)
    ctx_len = min(total_tokens - 1, CONTEXT_LENGTH)

    # Sequence stores ctx_len inputs plus the immediate next token for supervision
    sequence = tokens[:ctx_len + 1]

    # If the example was shorter than CONTEXT_LENGTH, pad the trailing target slot with 0
    if len(sequence) < ctx_len + 1:
        sequence = sequence + [0]

    pair_file = output_dir / f"pair_{idx:05d}.bin"

    with open(pair_file, 'wb') as f:
        f.write(struct.pack('HH', ctx_len, 1))
        f.write(struct.pack(f'{len(sequence)}I', *sequence))

    if (idx + 1) % 1000 == 0:
        print(f"  Saved {idx + 1}/{len(df_train)}")

print(f"
✓ Complete! Saved {len(df_train)} binary files")


## Step 8: Verify Binary Files

In [ ]:
# Read back first file to verify
test_file = output_dir / "pair_00000.bin"

with open(test_file, 'rb') as f:
    # Read header
    header = f.read(4)
    ctx_len, tgt_len = struct.unpack('HH', header)
    
    # Read tokens
    total_tokens = ctx_len + tgt_len
    tokens_bytes = f.read(4 * total_tokens)
    tokens = list(struct.unpack(f'{total_tokens}I', tokens_bytes))

print(f"Verification of {test_file.name}:")
print("="*60)
print(f"Header:")
print(f"  ctx_len: {ctx_len}")
print(f"  tgt_len: {tgt_len}")
print(f"\nTokens: {len(tokens)} total")
print(f"  First 10: {tokens[:10]}")
print(f"\nDecoded text:")
print(f"  {tokenizer.decode(tokens[:ctx_len])}")
print(f"\nMatches original: {tokens[:ctx_len] == df_train.iloc[0]['tokens']}")

## Step 9: Save Metadata

In [ ]:
import json

metadata = {
    'dataset': 'WikiSQL',
    'task': 'SQL generation',
    'num_examples': len(df_train),
    'context_length': CONTEXT_LENGTH,
    'vocab_size': 50257,
    'tokenizer': 'gpt2',
    'format': '[TASK: SQL] Table: columns | Question: question → SQL query',
    'avg_tokens': float(df_train['ctx_len'].mean()),
    'max_tokens': int(df_train['ctx_len'].max()),
    'min_tokens': int(df_train['ctx_len'].min())
}

metadata_file = output_dir / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print("Saved metadata:")
print(json.dumps(metadata, indent=2))

## Summary

In [ ]:
print("="*60)
print("SQL TRAINING DATA READY")
print("="*60)
print(f"Dataset:           WikiSQL")
print(f"Training examples: {len(df_train)}")
print(f"Context length:    {CONTEXT_LENGTH}")
print(f"Avg tokens/sample: {df_train['ctx_len'].mean():.1f}")
print(f"Files location:    {output_dir}")
print()
print("File format:")
print("  [uint16 ctx_len][uint16 tgt_len][ctx_len+tgt_len uint32 tokens]")
print()
print("C-Transformer will:")
print("  1. Read ctx_len from header")
print("  2. Load ctx_len + 1 tokens")
print("  3. input = tokens[0:ctx_len]")
print("  4. target = tokens[1:ctx_len+1] (next-token prediction)")
print("  5. Compute loss only over ctx_len positions")
print()
print(f"Expected training time: ~{len(df_train) * 10 // 3600}-{len(df_train) * 20 // 3600} hours")
print(f"  (depends on CPU and batch size)")